# Load CSVs (one-to-many) to Azure Cognitive Search

In this Jupyter Notebook, we create and run steps to index a CSV file in which each row is an individual and independent record/document. Each row then becomes searchable in Azure Cognitive Search.
The reference documentation can be found at [Indexing blobs and files to produce multiple search documents](https://learn.microsoft.com/en-us/azure/search/search-howto-index-one-to-many-blobs).

By default, an indexer will treat the contents of a blob or file as a single search document. If you want a more granular representation in a search index, you can set parsingMode values to create multiple search documents from one blob or file.

We are going to be using the same private Blob Storage account but a different container that has abstracts of 90k Medical publications about COVID-19 published in 2020-2022. This file is a subset of a much bigger dataset (770k articles) called CORD19 [HERE](https://github.com/allenai/cord19)


In [3]:
import os
import json
import requests
from dotenv import load_dotenv

load_dotenv("./.vscode/credentials.env")

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = "hack"

In [6]:
# Define the names for the data source, index and indexer
datasource_name = "cogsrch-datasource-esxp"
skillset_name = "cogsrch-skillset-esxp"
index_name = "cogsrch-index-sales-cs"
indexer_name = "cogsrch-index-sales-esxp"

In [4]:
# Setup the Payloads header
headers = {
    "Content-Type": "application/json",
    "api-key": os.environ["AZURE_SEARCH_KEY"],
}
params = {"api-version": os.environ["AZURE_SEARCH_API_VERSION"]}

## Create Data Source (Blob container with the Litcovid CSV data file)


In [8]:
# Create a data source

datasource_payload = {
    "name": datasource_name,
    "description": "Index Data Lake Gen 2.",
    "type": "adlsgen2",
    "credentials": {"connectionString": os.environ["ADLS_CONNECTION_STRING"]},
    "container": {"name": BLOB_CONTAINER_NAME, "query": "esxp"},
}
r = requests.put(
    os.environ["AZURE_SEARCH_ENDPOINT"] + "/datasources/" + datasource_name,
    data=json.dumps(datasource_payload),
    headers=headers,
    params=params,
)
print(r.status_code)
print(r.ok)

204
True


## Inspect CSV file so we can understand the column types before creating the Index


In our private dataset we have place a smaller version of the original the metadata.csv file in the cord19 dataset.
Let's see what the file looks like:


In [25]:
# Download the csv files to disk and inspect using pandas
import pandas as pd

remote_file_path = "./data/extract_esxp_offerings.csv"

In [30]:
metadata = pd.read_csv(remote_file_path, encoding="latin-1")
print("No. of lines:", metadata.shape[0])
# Service Name	Item Code	Service Type	Service Category	Service Family	Price	Currency	Price (USD)	Focus Area	Action Area	Primary Technology	Target Level	Maturity Level	Delivery Domain	Service Division	Content Language	Description	Data sheet Link	InternalComments	Duration

simple_schema = [
    "ServiceName",
    "ItemCode",
    "ServiceType",
    "ServiceCategory",
    "ServiceFamily",
    "Price",
    "PriceUSD",
    "Currency",
    "Description",
    "DatasheetLink",
    "InternalComments",
    "Duration",
]


def make_clickable(address):
    """Make the url clickable"""
    return '<a href="{0}">{0}</a>'.format(address)


def preview(text):
    """Show only a preview of the text data."""
    return text[:30] + "..."


format_ = {
    "ServiceName": preview,
    "ServiceCategory": preview,
    "ServiceFamily": preview,
    "DatasheetLink": make_clickable,
}

metadata[simple_schema].head().style.format(format_)

No. of lines: 1655


,ServiceName,ItemCode,ServiceType,ServiceCategory,ServiceFamily,Price,PriceUSD,Currency,Description,DatasheetLink,InternalComments,Duration
0,10 Additional Service Delivery...,SS1A,Service Delivery Management,Delivery Management...,Delivery Management...,300.15,220,CAD,Service Delivery Management to facilitate planning and implementation of support services.,nan,nan,1 Day
1,20 Additional Service Delivery...,SSA2,Service Delivery Management,Delivery Management...,Delivery Management...,272.86,200,CAD,Service Delivery Management to facilitate planning and implementation of support services.,nan,nan,1 Day
2,5 Additional Service Delivery ...,SS5A,Service Delivery Management,Delivery Management...,Delivery Management...,327.43,240,CAD,Service Delivery Management to facilitate planning and implementation of support services.,nan,nan,1 Day
3,ARR Problem Resolution As-Need...,SPAP,Problem Resolution Support,Reactive...,Reactive...,0,0,USD,"Assistance for problems with specific symptoms encountered while using Microsoft Azure, including troubleshooting specific problems, error messages or functionality that is not working as intended for Microsoft products.",nan,nan,0 Hour
4,ASfP Cloud Consult - Add On...,SAFL,Adoption Services,Proactive...,Optimize...,"2,728.60","2,000.00",CAD,"1-on-1 engagement with a Microsoft technical resource who provides you with best practice and architectural guidance of your deployment, migration, and implementations.","""https://microsoft.sharepoint-df.com/teams/IPCollateral/_layouts/DocIdRedir.aspx?ID=NNJFDHZT2Q5V-2-9681""",New IP request from v-nanlan; IP release date on 2/25/2018.,1 Day


## Create Skillset - Text Splitter, Language Detection

We will use cognitive services enrichment for spliting the text of each content field into chunks (pages) and for language detection. We should always split the text since we don't know how big the content of each row might be.


In [48]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "Splits Text and detect language",
    "skills": [
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 5000,  # 5000 is default
            "defaultLanguageCode": "en",
            "inputs": [{"name": "text", "source": "/document/description"}],
            "outputs": [{"name": "textItems", "targetName": "pages"}],
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ["COG_SERVICES_NAME"],
        "key": os.environ["COG_SERVICES_KEY"],
    },
}

r = requests.put(
    os.environ["AZURE_SEARCH_ENDPOINT"] + "/skillsets/" + skillset_name,
    data=json.dumps(skillset_payload),
    headers=headers,
    params=params,
)
print(r.status_code)
print(r.ok)

204
True


In [58]:
index_payload = {
    "name": index_name,
    "fields": [
        {
            "name": "id",
            "type": "Edm.String",
            "key": "true",
            "searchable": "false",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "ServiceName",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "true",
            "sortable": "false",
        },
        {
            "name": "ServiceType",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false",
        },
        {
            "name": "ServiceCategory",
            "type": "Edm.String",
            "searchable": "false",
            "retrievable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false",
        },
        {
            "name": "ServiceFamily",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "Price",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "PriceUSD",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "Currency",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "Description",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "url",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "InternalComments",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "Duration",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "FocusArea",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "ActionArea",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "PrimaryTechnology",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "TargetLevel",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "MaturityLevel",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "DeliveryDomain",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "ServiceDivision",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "ContentLanguage",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "vectorized",
            "type": "Edm.Boolean",
            "searchable": "false",
            "retrievable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false",
        },
        {
            "name": "metadata_storage_name",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false",
        },
        {
            "name": "metadata_storage_path",
            "type": "Edm.String",
            "searchable": "false",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "metadata_storage_last_modified",
            "type": "Edm.DateTimeOffset",
            "searchable": "false",
            "retrievable": "false",
            "filterable": "false",
            "sortable": "false",
        },
        {
            "name": "chunks",
            "type": "Collection(Edm.String)",
            "searchable": "false",
            "retrievable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false",
        },
        {
            "name": "Vector",
            "type": "Collection(Edm.Single)",
            "searchable": "true",
            "retrievable": "true",
            "dimensions": 1536,
            "vectorSearchConfiguration": "vectorConfig",
        },
    ],
    "vectorSearch": {
        "algorithmConfigurations": [{"name": "vectorConfig", "kind": "hnsw"}]
    },
    "profiles": [
        {
            "name": "vectorConfig-profile",
            "algorithm": "vectorConfig",
            "vectorizer": "vectorizer-config",
        }
    ],
    "vectorizers": [
        {
            "name": "vectorizer-config",
            "kind": "azureOpenAI",
            "azureOpenAIParameters": {
                "resourceUri": os.environ["AZURE_OPENAI_ENDPOINT"],
                "deploymentId": "text-embedding-ada-002",
                "apiKey": os.environ["AZURE_OPENAI_API_KEY"],
            },
        }
    ],
    "semantic": {
        "configurations": [
            {
                "name": "esxp-semantic-config",
                "prioritizedFields": {
                    "titleField": {"fieldName": "ServiceName"},
                    "prioritizedContentFields": [{"fieldName": "Description"}],
                },
            }
        ]
    },
}
r = requests.put(
    os.environ["AZURE_SEARCH_ENDPOINT"] + "/indexes/" + index_name,
    data=json.dumps(index_payload),
    headers=headers,
    params=params,
)
print(r.status_code)
print(r.ok)
print(r.text)

201
True
{"@odata.context":"https://cog-search-zsqq6nulv33fy.search.windows.net/$metadata#indexes/$entity","@odata.etag":"\"0x8DBED114713496C\"","name":"cogsrch-index-sales-cs","defaultScoringProfile":null,"fields":[{"name":"id","type":"Edm.String","searchable":false,"filterable":false,"retrievable":true,"sortable":false,"facetable":false,"key":true,"indexAnalyzer":null,"searchAnalyzer":null,"analyzer":null,"normalizer":null,"dimensions":null,"vectorSearchConfiguration":null,"synonymMaps":[]},{"name":"ServiceName","type":"Edm.String","searchable":true,"filterable":true,"retrievable":true,"sortable":false,"facetable":false,"key":false,"indexAnalyzer":null,"searchAnalyzer":null,"analyzer":null,"normalizer":null,"dimensions":null,"vectorSearchConfiguration":null,"synonymMaps":[]},{"name":"ServiceType","type":"Edm.String","searchable":true,"filterable":false,"retrievable":true,"sortable":false,"facetable":false,"key":false,"indexAnalyzer":null,"searchAnalyzer":null,"analyzer":null,"normali

## Create and Run the Indexer - (runs the pipeline)

To create one-to-many indexers with CSV blobs, create or update an indexer definition with the delimitedText parsing mode


In [59]:
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "schedule": {"interval": "PT2H"},
    "fieldMappings": [
        {"sourceFieldName": "ItemCode", "targetFieldName": "id"},
        {"sourceFieldName": "ServiceName", "targetFieldName": "ServiceName"},
        {"sourceFieldName": "ServiceType", "targetFieldName": "ServiceType"},
        {"sourceFieldName": "ServiceCategory", "targetFieldName": "ServiceCategory"},
        {"sourceFieldName": "ServiceFamily", "targetFieldName": "ServiceFamily"},
        {"sourceFieldName": "Price", "targetFieldName": "Price"},
        {"sourceFieldName": "PriceUSD", "targetFieldName": "PriceUSD"},
        {"sourceFieldName": "Currency", "targetFieldName": "Currency"},
        {"sourceFieldName": "FocusArea", "targetFieldName": "FocusArea"},
        {"sourceFieldName": "ActionArea", "targetFieldName": "ActionArea"},
        {
            "sourceFieldName": "PrimaryTechnology",
            "targetFieldName": "PrimaryTechnology",
        },
        {"sourceFieldName": "TargetLevel", "targetFieldName": "TargetLevel"},
        {"sourceFieldName": "MaturityLevel", "targetFieldName": "MaturityLevel"},
        {"sourceFieldName": "DeliveryDomain", "targetFieldName": "DeliveryDomain"},
        {"sourceFieldName": "ServiceDivision", "targetFieldName": "ServiceDivision"},
        {"sourceFieldName": "ContentLanguage", "targetFieldName": "ContentLanguage"},
        {"sourceFieldName": "Description", "targetFieldName": "Description"},
        {"sourceFieldName": "DatasheetLink", "targetFieldName": "url"},
        {"sourceFieldName": "InternalComments", "targetFieldName": "InternalComments"},
        {"sourceFieldName": "Duration", "targetFieldName": "Duration"},
        {
            "sourceFieldName": "DatasheetLink",
            "targetFieldName": "metadata_storage_path",
        },
    ],
    "outputFieldMappings": [
        {"sourceFieldName": "/document/pages/*", "targetFieldName": "chunks"}
    ],
    "parameters": {
        "configuration": {
            "dataToExtract": "contentAndMetadata",
            "parsingMode": "delimitedText",
            "firstLineContainsHeaders": True,
            "delimitedTextDelimiter": ",",
        }
    },
}


r = requests.put(
    os.environ["AZURE_SEARCH_ENDPOINT"] + "/indexers/" + indexer_name,
    data=json.dumps(indexer_payload),
    headers=headers,
    params=params,
)


print(r.status_code)


print(r.ok)
print(r.text)

204
True



In [60]:
# Optionally, get indexer status to confirm that it's running
try:
    r = requests.get(
        os.environ["AZURE_SEARCH_ENDPOINT"] + "/indexers/" + indexer_name + "/status",
        headers=headers,
        params=params,
    )
    # pprint(json.dumps(r.json(), indent=1))
    print(r.status_code)
    print("Status:", r.json().get("lastResult").get("status"))
    print("Items Processed:", r.json().get("lastResult").get("itemsProcessed"))
    print(r.ok)

except Exception as e:
    print("Wait a few seconds until the process starts and run this cell again.")

200
Status: success
Items Processed: 1655
True


**When the indexer finishes running we will have all 90,000 rows indexed properly as separate documents in our Search Engine!.**


## Creation of its corresponding vector-based index


In [7]:
index_name = "adlsgen2-index"
index_payload = {
    "name": "adlsgen2-index-vector",
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true"},
        {
            "name": "title",

            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
        },
        {
            "name": "chunk",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
        },
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "searchable": "true",
            "retrievable": "true",
            "dimensions": 1536,
            "vectorSearchConfiguration": "vectorConfig",
        },
        {
            "name": "name",

            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false",
        },
        {
            "name": "location",

            "type": "Edm.String",
            "searchable": "false",
            "retrievable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false",
        },
    ],
    "vectorSearch": {
        "algorithmConfigurations": [{"name": "vectorConfig", "kind": "hnsw"}]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {"fieldName": "title"},
                    "prioritizedContentFields": [{"fieldName": "chunk"}],
                    "prioritizedKeywordsFields": [],
                },
            }
        ]
    },
}



r = requests.put(
    os.environ["AZURE_SEARCH_ENDPOINT"] + "/indexes/" + index_name + "-vector",
    data=json.dumps(index_payload),
    headers=headers,
    params=params,
)


print(r.status_code)


print(r.ok)

201
True


# Reference

- https://learn.microsoft.com/en-us/azure/search/search-howto-index-csv-blobs
- https://learn.microsoft.com/en-us/azure/search/knowledge-store-create-rest


# NEXT

Now that we have two separate text-based indexes loaded with two different types of information and its correspongind vector-based indexes, In the next notebook 3, we will do a Multi-Index query, sort the results based on the reranker semantic score of Azure Search, and then use OpenAI to understand this results and give the best answer possible
